In [1]:
import pandas as pd
import numpy as np

# 1

In [110]:
df = pd.read_csv('online-retail.csv', sep=';', parse_dates=['InvoiceDate'])
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
541909,581588,22380,TOY TIDY SPACEBOY,9,2011-12-09 12:52:00,2.10,12594.0,Italy


# 2, 3

In [111]:
df.shape

(541911, 8)

# 4

In [112]:
min(df.columns)

'Country'

# 5

In [116]:
sum(df.isnull().sum() > 0)

2

# 6

In [117]:
df.CustomerID.isnull().sum()

135080

# 7

In [120]:
df.InvoiceDate.agg(lambda x: x.year).agg(['min', 'max'])

min    2010
max    2011
Name: InvoiceDate, dtype: int64

# 8

In [121]:
df.UnitPrice.agg(['min', 'max'])

min   -11062.06
max    38970.00
Name: UnitPrice, dtype: float64

# 9

In [402]:
np.sort(df[df.UnitPrice < 0].Description.unique())[0]

'Adjust bad debt'

# 10

In [139]:
df[df.UnitPrice == 0].shape[0]

2515

In [157]:
df.UnitPrice.isnull().sum()

0

# 11

In [158]:
df_correct = df[(df.UnitPrice > 0) & (df.Quantity > 0)].dropna()
df_correct.shape[0]

397886

# 12

In [166]:
df_correct['Price'] = df_correct.UnitPrice * df_correct.Quantity
df_correct.Price.agg(lambda x: [round(min(x)), round(max(x))])

[0, 168470]

# 13

In [173]:
round(df_correct.groupby('InvoiceNo')['Price'].sum().sort_values(ascending=False).head(3))

InvoiceNo
581483    168470.0
541431     77184.0
556444     38970.0
Name: Price, dtype: float64

# 14

In [177]:
df_correct.groupby('Description')['Price'].sum().sort_values(ascending=False).index[0]

'PAPER CRAFT , LITTLE BIRDIE'

# 15

In [183]:
purchases = df_correct.groupby('InvoiceNo').agg({'InvoiceDate': 'min', 'Price': 'sum', 'CustomerID': 'first', 
                                                 'Country': 'first'}).reset_index().sort_values(by=['InvoiceDate'])
purchases.shape[0]

18533

# 16

In [192]:
m_first = pd.merge(purchases.groupby('CustomerID').agg({'InvoiceDate': 'min'}).reset_index(), 
                   purchases, on=['CustomerID', 'InvoiceDate'])
[round(np.mean(m_first.Price.values)), round(np.mean(purchases.Price))]

[425.0, 481]

# 17

In [207]:
purchases['Day'] = purchases['InvoiceDate'].apply(lambda x: x.strftime('%A'))
purchases.groupby('Day')['Price'].count().sort_values(ascending=False).index[0]

'Thursday'

# 18

In [215]:
purchases['Month'] = purchases['InvoiceDate'].apply(lambda x: x.month)
purchases['Year'] = purchases['InvoiceDate'].apply(lambda x: x.year)
purchases.groupby(['Month', 'Year'])['Price'].count().sort_values(ascending=False).head(1).index[0]

(11, 2011)

# 19

In [407]:
df_correct['Month'] = df_correct['InvoiceDate'].apply(lambda x: x.month)
df_correct['Year'] = df_correct['InvoiceDate'].apply(lambda x: x.year)
increase_march = df_correct[(df_correct.Year == 2011) & (df_correct.Month == 3)].groupby(['Country', 'Month'])['Price'].count().reset_index()
increase_september = df_correct[(df_correct.Year == 2011) & (df_correct.Month == 9)].groupby(['Country', 'Month'])['Price'].count().reset_index()
increase = pd.merge(increase_march, increase_september, on='Country')
increase['percent'] = ((increase.Price_y - increase.Price_x) / increase.Price_x * 100).apply(lambda x: round(x))
increase.sort_values(by=['percent'], ascending=False).reset_index(drop=True).head(1)[['Country', 'percent']]

,Country,percent
0,Norway,778


# 20

In [264]:
purchases.head(2)

,InvoiceNo,InvoiceDate,Price,CustomerID,Country,Day,Month,Year
0,536365,2010-12-01 08:26:00,139.12,17850.0,United Kingdom,Wednesday,12,2010
1,536366,2010-12-01 08:28:00,22.20,17850.0,United Kingdom,Wednesday,12,2010


In [272]:
group_id_country = purchases.groupby(['CustomerID', 'Country'])['Price'].count().reset_index()
group_id = group_id_country.groupby('CustomerID')['Country'].count().reset_index()
group_id[group_id.Country > 1].shape[0]

8

# 21

In [400]:
from itertools import combinations
Italy = df_correct[df_correct.Country == 'Italy']
purchase_set = Italy.groupby('InvoiceNo')['Description'].apply(lambda x: set(x)).reset_index()
purchase_set.Description
a = purchase_set.Description.apply(lambda x: [', '.join(sorted(y)) for y in list(combinations(x,2))])
c = pd.Series([x for y in a.values for x in y])
c.groupby(c).size().nlargest(1)

TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY    6
dtype: int64